In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [2]:
import torch
from haystack.utils import clean_wiki_text
from haystack.utils import convert_files_to_docs
from haystack.utils import fetch_archive_from_http
from haystack.utils import print_answers
from haystack.nodes import FARMReader, TransformersReader

f:\Asha\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# doc_dir = "/content/drive/MyDrive/Asha/text-as-page"
doc_dir = "F:/Asha/text-as-page"
# C:\Users\DELL\Desktop\Asha\text-as-page


# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.

docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-10.txt


INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-11.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-12.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-13.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-14.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-15.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-16.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-17.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-18.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-19.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-page-20.txt
INFO - haystack.utils.preprocessing -  Converting F:\Asha\text-as-page\asha_ncd-

In [ ]:
# import os
# from subprocess import Popen, PIPE, STDOUT

# # es_server = Popen(
# #     ["G:/models/elastic-search/elasticsearch-8.10.4/binelasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)
# # )

# es_server = Popen(
#     ["H:/models/elastic-test/elasticsearch-8.10.4-windows-x86_64/elasticsearch-8.10.4/bin/elasticsearch.bat"], stdout=PIPE, stderr=STDOUT
# )


# ! sleep 30

'sleep' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
from haystack.document_stores import ElasticsearchDocumentStore
# ========================================
# document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document2")
document_store = ElasticsearchDocumentStore(
    host="localhost",  # Use the HTTPS scheme
    username="elastic",
    password="GPvaCsQYvSnI_EkU3vLV",
    index="full-clean-1",
    verify_certs=True,  # Set this to True if you want to verify the SSL certificate
    # ssl_show_warn=False,  # Set this to False if you don't want to show SSL warnings
)

WARNING - haystack.document_stores.elasticsearch.base -  This ElasticsearchDocumentStore has been built for Elasticsearch 7, but the detected version of the Elasticsearch server is 8.10.4. Unexpected behaviors or errors may occur due to version incompatibility.


In [5]:
document_store.write_documents(docs)

In [6]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [7]:
saved_model_dir = "F:/haystack-roberta-base-squad2"
reader = FARMReader(model_name_or_path=saved_model_dir, use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'F:/haystack-roberta-base-squad2' (Roberta)
f:\Asha\venv\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO - haystack.modeling.model.language_model -  Loaded 'F:/haystack-roberta-base-squad2' (Roberta model) from local file system.
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a proble

In [ ]:
# reader.save(saved_model_dir)

In [8]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [9]:
prediction = pipe.run(
    query="Risk Factors for Hypertension", params={"Retriever": {"top_k": 2}, "Reader": {"top_k": 10}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:07<00:00,  7.00s/ Batches]


In [10]:
prediction


{'query': 'Risk Factors for Hypertension',
 'no_ans_gap': -6.149246454238892,
 'answers': [<Answer {'answer': 'Unhealthy diet', 'type': 'extractive', 'score': 0.04389713332056999, 'context': 'owing are some common factors that can lead to high blood pressure \nUnhealthy diet - A diet especially high in salt, fat and low in vegetables/fruits.', 'offsets_in_document': [{'start': 980, 'end': 994}], 'offsets_in_context': [{'start': 68, 'end': 82}], 'document_ids': ['afdf27663c0151f05a751e903e66bacb'], 'meta': {'name': 'asha_ncd-page-35.txt'}}>,
  <Answer {'answer': 'high blood pressure', 'type': 'extractive', 'score': 0.03751356154680252, 'context': 's we grow older, there is an   increase in the risk of developing high blood pressure, (Hypertension), high blood   sugar levels, (Diabetes), high lev', 'offsets_in_document': [{'start': 430, 'end': 449}], 'offsets_in_context': [{'start': 66, 'end': 85}], 'document_ids': ['71d309867989bc27b313a15715292426'], 'meta': {'name': 'asha_ncd-page-16.

In [ ]:
prediction


{'query': 'Risk Factors for Hypertension',
 'no_ans_gap': -5.296202659606934,
 'answers': [<Answer {'answer': 'Unhealthy diet', 'type': 'extractive', 'score': 0.05551372468471527, 'context': ' are some common factors that can lead to high blood pressure \nï¿½  Unhealthy diet - A diet especially high in salt, fat and low in vegetables/fruits.', 'offsets_in_document': [{'start': 985, 'end': 999}], 'offsets_in_context': [{'start': 68, 'end': 82}], 'document_ids': ['a95f15b0d9563a4201d656cb18161689'], 'meta': {'name': 'asha_ncd-page-35.txt'}}>,
  <Answer {'answer': 'ï¿', 'type': 'extractive', 'score': 0.03754296153783798, 'context': 'e following are some common factors that can lead to high blood pressure \nï¿½  Unhealthy diet - A diet especially high in salt, fat and low in vegetab', 'offsets_in_document': [{'start': 980, 'end': 982}], 'offsets_in_context': [{'start': 74, 'end': 76}], 'document_ids': ['a95f15b0d9563a4201d656cb18161689'], 'meta': {'name': 'asha_ncd-page-35.txt'}}>,
  <Answ

In [15]:
prediction = pipe.run(
    query="Cancer", params={"Retriever": {"top_k": 2}, "Reader": {"top_k": 10}}
)
prediction

Inferencing Samples: 100%|██████████| 1/1 [00:05<00:00,  5.17s/ Batches]


{'query': 'Cancer',
 'no_ans_gap': -5.461440086364746,
 'answers': [<Answer {'answer': 'ï¿', 'type': 'extractive', 'score': 0.043330539017915726, 'context': 'he health system and   encourage the community to help them as required. \nï¿½  Encourage the patients undergoing cancer treatment to have fresh,   hyg', 'offsets_in_document': [{'start': 721, 'end': 723}], 'offsets_in_context': [{'start': 74, 'end': 76}], 'document_ids': ['868a98cd64fcdd3393975a845d527a02'], 'meta': {'name': 'asha_ncd-page-56.txt'}}>,
  <Answer {'answer': 'COPD', 'type': 'extractive', 'score': 0.02659785747528076, 'context': 'ypertension Diabetes Oral   Cancer  Breast   Cancer   Cervical   Cancer  COPD   (Respiratory   Disorders) \nCondition Date of   Diagnosis  Treatment In', 'offsets_in_document': [{'start': 526, 'end': 530}], 'offsets_in_context': [{'start': 73, 'end': 77}], 'document_ids': ['943ad08ddf65b8f4a2a7006d2ae1a6bf'], 'meta': {'name': 'asha_ncd-page-66.txt'}}>,
  <Answer {'answer': 'Hypertension', 't

In [ ]:
qry = "lump in breast"
prediction = pipe.run(
    query=qry, params={"Retriever": {"top_k": 2}, "Reader": {"top_k": 10}}
)
prediction

Inferencing Samples: 100%|██████████| 1/1 [00:06<00:00,  6.15s/ Batches]


{'query': 'lump in breast',
 'no_ans_gap': -0.9495975971221924,
 'answers': [<Answer {'answer': 'Shortness of breath', 'type': 'extractive', 'score': 0.3197242021560669, 'context': 'hese Symptoms \nB1: Women and Men  Yes/No  B2: Women only  Yes/No \nShortness of breath   Lump in the breast \nCoughing more than 2 weeks  Blood stained ', 'offsets_in_document': [{'start': 175, 'end': 194}], 'offsets_in_context': [{'start': 66, 'end': 85}], 'document_ids': ['9419976ac5ff99e98646a200d99c0333'], 'meta': {'name': 'asha_ncd-page-68.txt'}}>,
  <Answer {'answer': 'ï¿½  Lump or thickening', 'type': 'extractive', 'score': 0.2009580284357071, 'context': 'h nipples. \nï¿½  Puckering or dimpling of skin of the breasts. \nï¿½  Lump or thickening in the breast. \nï¿½  Constant pain in the breast or armpit. \nW', 'offsets_in_document': [{'start': 434, 'end': 457}], 'offsets_in_context': [{'start': 64, 'end': 87}], 'document_ids': ['578d364f89ccf698159c10991eee0a1d'], 'meta': {'name': 'asha_ncd-page-51.t

In [14]:
qry = "diabetes"
prediction = pipe.run(
    query=qry, params={"Retriever": {"top_k": 2}, "Reader": {"top_k": 10}}
)
prediction

Inferencing Samples: 100%|██████████| 1/1 [00:04<00:00,  4.73s/ Batches]


{'query': 'diabetes',
 'no_ans_gap': 5.494028568267822,
 'answers': [<Answer {'answer': 'Type 1, Type 2 and Gestational Diabetes', 'type': 'extractive', 'score': 0.35358190536499023, 'context': '  you will learn about: \n Different Types of Diabetes, (Type 1, Type 2 and Gestational Diabetes). \n Risk Factors for Type 2 Diabetes. \n Common Signs a', 'offsets_in_document': [{'start': 586, 'end': 625}], 'offsets_in_context': [{'start': 56, 'end': 95}], 'document_ids': ['d8dce559ba8874cfa42757da663065c2'], 'meta': {'name': 'asha_ncd-page-42.txt'}}>,
  <Answer {'answer': 'Type 2 Diabetes', 'type': 'extractive', 'score': 0.31855764985084534, 'context': 'Types of Diabetes  What is it?  Who gets it? \nType 2 Diabetes   This is the  most common   type  of diabetes. The body   produces some insulin, but   ', 'offsets_in_document': [{'start': 46, 'end': 61}], 'offsets_in_context': [{'start': 46, 'end': 61}], 'document_ids': ['f2834335a5fce97ffd4c78cdd9342636'], 'meta': {'name': 'asha_ncd-page-43.

In [ ]:
import re

retJson = {}
retJson['res'] = []

for ans in prediction['answers']:
  scr =ans.score
  answer = ans.answer.replace("/square6", "#").replace("\n", "")
  context = re.split("[\n>]", ans.context.replace("/square6", "># ").replace("\n", ""))
  retJson['res'].append({'score': scr, "answer": answer,  "context":context})

In [ ]:
retJson

{'res': [{'score': 0.1522972285747528,
   'answer': 'smoking and smokeless forms',
   'context': ['at are the Various Forms in which Tobacco is Used? There are smoking and smokeless forms of tobacco. ï¿½ ï¿½ Smoking tobacco can be in the form of c']},
  {'score': 0.023050582036376,
   'answer': '2',
   'context': ['bacco and   alcohol to their health and the health of those around them.  2. Improve understanding of the money spent on tobacco and alcohol use and ']},
  {'score': 0.018003644421696663,
   'answer': '2. Improve understanding of the money spent on tobacco and alcohol use and   the likely costs of treatment needed if they were to fall ill.  3',
   'context': ['.  2. Improve understanding of the money spent on tobacco and alcohol use and   the likely costs of treatment needed if they were to fall ill.  3. E']},
  {'score': 0.01641761139035225,
   'answer': '4. Work with individuals who use tobacco and alcohol and motivate them to   quit these habits by explaining the harmful 